This is to import all the modules used

In [1]:
import csv as csv
import numpy as np
import pandas
import matplotlib.pyplot as plt
import pylab
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold




This is to import both the test and train data

In [2]:

#load in csv file train data
data = pandas.read_csv('train.csv', header = 0)

#load in csv file test data
data_test = pandas.read_csv('test.csv', header = 0)


Exploratory analysis on train data - not to be used when running the model

In [12]:


table = pandas.DataFrame.pivot_table(data, index = ['Sex'], columns = 'Survived', aggfunc = np.size)

print table

         PassengerId        Pclass          Name           Age         SibSp  \
Survived           0      1      0      1      0      1      0      1      0   
Sex                                                                            
female          81.0  233.0   81.0  233.0   81.0  233.0   81.0  233.0   81.0   
male           468.0  109.0  468.0  109.0  468.0  109.0  468.0  109.0  468.0   

                 Parch        Ticket          Fare         Cabin         \
Survived      1      0      1      0      1      0      1      0      1   
Sex                                                                       
female    233.0   81.0  233.0   81.0  233.0   81.0  233.0   81.0  233.0   
male      109.0  468.0  109.0  468.0  109.0  468.0  109.0  468.0  109.0   

         Embarked         
Survived        0      1  
Sex                       
female       81.0  233.0  
male        468.0  109.0  


In [13]:
data.describe()

/Library/Python/2.7/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


Change some of the string variables to numerical variables

In [15]:
#change the Sex variable to Gender
data['Gender']=4
data['Gender'] = data['Sex'].map( {'female':1, 'male':0}).astype(int)

#Fill in missing ages and call AgeFill
data['AgeFill']=data['Age']
median_ages = np.zeros((2,3))

for i in range(0,2):
	for j in range(0,3):
		median_ages[i,j] = data[(data['Gender'] == i ) & (data['Pclass'] ==j+1)]['Age'].dropna().median()


for i in range(0,2):
	for j in range(0,3):
		data.loc[ (data.Age.isnull() & (data['Gender']==i) & (data['Pclass']==j+1)), 'AgeFill']=median_ages[i,j]

#print data[data['Age'].isnull()][['Gender', 'Pclass', 'AgeFill', 'Age']].head(10)

data['Agemissing']=pandas.isnull(data.Age).astype(int)

data['FamilySize'] = data['Parch'] + data['SibSp']

